In [1]:
import urllib, requests
import sqlite3 as sql
import json
from bs4 import BeautifulSoup
import time
import xml.etree.cElementTree as et
from lxml import etree
from lxml.etree import fromstring

In [2]:
def getResponse(pageNum):
    url = "http://www.bokjiro.go.kr/openapi/nwel/getDisConvFaclList"
    def_params = {
        "crtiKey":"kVk1lHt6K5op%2ByDKNNCgEOdOfk36JEaU%2Ff4p461vk2H5X726llX6wDaqi9%2BBLfCsaITLqAE5IxiGuwRJ6v6XgA%3D%3D",
        "pageNo": pageNum,
        "startPage":1,
        "numOfRows":999,
        "pageSize":999,
        "siDoNm":"서울특별시",
#         "_returnType":"json"
    }

#     def_params["stationName"] = "서울특별시"
    def_params["crtiKey"]  = urllib.parse.unquote(def_params["crtiKey"] )
    resp = requests.get(url,params=def_params)
    return resp

In [3]:
def getInfoResponse(wfcltId):
    url = "http://www.bokjiro.go.kr/openapi/nwel/getDisConvFaclList"
    def_params = {
        "crtiKey":"kVk1lHt6K5op%2ByDKNNCgEOdOfk36JEaU%2Ff4p461vk2H5X726llX6wDaqi9%2BBLfCsaITLqAE5IxiGuwRJ6v6XgA%3D%3D",
        "wfcltId":wfcltId
    }
    def_params["crtiKey"]  = urllib.parse.unquote(def_params["crtiKey"] )
    resp = requests.get(url,params=def_params)
    return resp

In [2]:
conn = sql.connect('disabled_facilities.db')
cur = conn.cursor()

In [5]:
# cur.execute("DROP TABLE facilities")
cur.executescript(
    """
    CREATE TABLE facilities(
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    faclInfId INTEGER,
    faclLat FLOAT,
    faclLng FLOAT,
    faclNm TEXT,
    faclTyCd TEXT(6),
    lcMnad TEXT,
    wfcltId TEXT,
    evalInfo TEXT
    );
    """
)

In [6]:
a = getResponse(1).text

In [5]:
class XmlDictConfig(dict):
    '''
    Example usage:

    >>> tree = ElementTree.parse('your_file.xml')
    >>> root = tree.getroot()
    >>> xmldict = XmlDictConfig(root)

    Or, if you want to use an XML string:

    >>> root = ElementTree.XML(xml_string)
    >>> xmldict = XmlDictConfig(root)

    And then use xmldict for what it is... a dict.
    '''
    def __init__(self, parent_element):
        if parent_element.items():
            self.update(dict(parent_element.items()))
        for element in parent_element:
            if element:
                # treat like dict - we assume that if the first two tags
                # in a series are different, then they are all different.
                if len(element) == 1 or element[0].tag != element[1].tag:
                    aDict = XmlDictConfig(element)
                # treat like list - we assume that if the first two tags
                # in a series are the same, then the rest are the same.
                else:
                    # here, we put the list in dictionary; the key is the
                    # tag name the list elements all share in common, and
                    # the value is the list itself 
                    aDict = {element[0].tag: XmlListConfig(element)}
                # if the tag has attributes, add those to the dict
                if element.items():
                    aDict.update(dict(element.items()))
                self.update({element.tag: aDict})
            # this assumes that if you've got an attribute in a tag,
            # you won't be having any text. This may or may not be a 
            # good idea -- time will tell. It works for the way we are
            # currently doing XML configuration files...
            elif element.items():
                self.update({element.tag: dict(element.items())})
            # finally, if there are no child tags and no attributes, extract
            # the text
            else:
                self.update({element.tag: element.text})

In [6]:
parser = etree.XMLParser(ns_clean=True, recover=True, encoding='utf-8')

In [8]:
xmlObj = etree.fromstring(a.encode('utf-8'), parser=parser)

NameError: name 'a' is not defined

In [10]:
xmlRoot = etree.ElementTree(xmlObj).getroot()

In [11]:
etree.dump(xmlRoot)

<facInfoList>
  <numOfRows>999</numOfRows>
  <pageNo>1</pageNo>
  <resultCode>000</resultCode>
  <resultMessage>조회 성공</resultMessage>
  <servList>
    <estbDate>19850219</estbDate>
    <faclInfId>20140610062</faclInfId>
    <faclLat>37.562725711470605</faclLat>
    <faclLng>127.03256993326957</faclLng>
    <faclNm>(구)왕십리클리닉빌딩</faclNm>
    <faclRprnNm>유삼석</faclRprnNm>
    <faclTyCd>UC0K02</faclTyCd>
    <lcMnad>서울특별시 성동구 왕십리로 333</lcMnad>
    <salStaDivCd>Y</salStaDivCd>
    <salStaNm>영업</salStaNm>
    <wfcltId>0000073296</wfcltId>
  </servList>
  <servList>
    <estbDate>20010316</estbDate>
    <faclInfId>20140615291</faclInfId>
    <faclLat>37.47798704381922</faclLat>
    <faclLng>126.95328960225815</faclLng>
    <faclNm>(길성아파트)하이텔하우스</faclNm>
    <faclRprnNm>김은기</faclRprnNm>
    <faclTyCd>UC0U01</faclTyCd>
    <lcMnad>서울특별시 관악구 관악로12길 16</lcMnad>
    <salStaDivCd>Y</salStaDivCd>
    <salStaNm>영업</salStaNm>
    <wfcltId>0000079520</wfcltId>
  </servList>
  <servList>
    <estbDate>200

In [11]:
l = xmlObj.findall('./servList')

In [7]:
# query = "INSERT INTO facilities (estbDate, faclInfId,faclLat,faclLng,faclNm,faclRprnNm,faclTyCd,lcMnad,salStaDivCd,salStaNm,wfcltId) \
#         VALUES (XmlDictConfig(i)['estbDate'], XmlDictConfig(i)['faclInfId'], XmlDictConfig(i)['faclLat'], XmlDictConfig(i)['faclLng'], XmlDictConfig(i)['faclNm'], \
#         XmlDictConfig(i)['faclRprnNm'], XmlDictConfig(i)['faclTyCd'], XmlDictConfig(i)['lcMnad'], XmlDictConfig(i)['salStaNm'], XmlDictConfig(i)['salStaNm']);"
query = "INSERT INTO facilities (faclInfId,faclLat,faclLng,faclNm,faclTyCd,lcMnad,wfcltId,evalInfo) VALUES (?,?,?,?,?,?,?,?)"
num = 9
# query = "INSERT INTO facilities (faclInfId,faclLat,faclLng,faclNm,faclTyCd,lcMnad,wfcltId) VALUES (?,?,?,?,?,?,?)"

for j in range(10, 20):
    num+=1
    ctr = 0
    xml = getResponse(j).text
    xmlObj = etree.fromstring(xml.encode('utf-8'), parser=parser)
    xmlRoot = etree.ElementTree(xmlObj).getroot()
    l = xmlObj.findall('./servList')
    
    for i in l:
    #     print(XmlDictConfig(i))
        dic = dict(XmlDictConfig(i))
        if dic['salStaDivCd']=='Y' and dic['salStaNm']=='영업':

            b = getInfoResponse(dic['wfcltId']).text
            xmlInfoObj = etree.fromstring(b.encode('utf-8'), parser=parser)
            xmlInfoRoot = etree.ElementTree(xmlInfoObj).getroot()
            l2 = xmlInfoObj.find('.//evalInfo').text

            cur.execute(query, (dic['faclInfId'], dic['faclLat'], dic['faclLng'], dic['faclNm'], dic['faclTyCd'], dic['lcMnad'], dic['wfcltId'], l2))
            ctr+=1
            print(num, ctr)
            time.sleep(1)

/Users/hanhyejeong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.


10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
10 16
10 17
10 18
10 19
10 20
10 21
10 22
10 23
10 24
10 25
10 26
10 27
10 28
10 29
10 30
10 31
10 32
10 33
10 34
10 35
10 36
10 37
10 38
10 39
10 40
10 41
10 42
10 43
10 44
10 45
10 46
10 47
10 48
10 49
10 50
10 51
10 52
10 53
10 54
10 55
10 56
10 57
10 58
10 59
10 60
10 61
10 62
10 63
10 64
10 65
10 66
10 67
10 68
10 69
10 70
10 71
10 72
10 73
10 74
10 75
10 76
10 77
10 78
10 79
10 80
10 81
10 82
10 83
10 84
10 85
10 86
10 87
10 88
10 89
10 90
10 91
10 92
10 93
10 94
10 95
10 96
10 97
10 98
10 99
10 100
10 101
10 102
10 103
10 104
10 105
10 106
10 107
10 108
10 109
10 110
10 111
10 112
10 113
10 114
10 115
10 116
10 117
10 118
10 119
10 120
10 121
10 122
10 123
10 124
10 125
10 126
10 127
10 128
10 129
10 130
10 131
10 132
10 133
10 134
10 135
10 136
10 137
10 138
10 139
10 140
10 141
10 142
10 143
10 144
10 145
10 146
10 147
10 148
10 149
10 150
10 151
10 152
10 153
10 154
10 155
10 156
10 157
10 158
10

11 204
11 205
11 206
11 207
11 208
11 209
11 210
11 211
11 212
11 213
11 214
11 215
11 216
11 217
11 218
11 219
11 220
11 221
11 222
11 223
11 224
11 225
11 226
11 227
11 228
11 229
11 230
11 231
11 232
11 233
11 234
11 235
11 236
11 237
11 238
11 239
11 240
11 241
11 242
11 243
11 244
11 245
11 246
11 247
11 248
11 249
11 250
11 251
11 252
11 253
11 254
11 255
11 256
11 257
11 258
11 259
11 260
11 261
11 262
11 263
11 264
11 265
11 266
11 267
11 268
11 269
11 270
11 271
11 272
11 273
11 274
11 275
11 276
11 277
11 278
11 279
11 280
11 281
11 282
11 283
11 284
11 285
11 286
11 287
11 288
11 289
11 290
11 291
11 292
11 293
11 294
11 295
11 296
11 297
11 298
11 299
11 300
11 301
11 302
11 303
11 304
11 305
11 306
11 307
11 308
11 309
11 310
11 311
11 312
11 313
11 314
11 315
11 316
11 317
11 318
11 319
11 320
11 321
11 322
11 323
11 324
11 325
11 326
11 327
11 328
11 329
11 330
11 331
11 332
11 333
11 334
11 335
11 336
11 337
11 338
11 339
11 340
11 341
11 342
11 343
11 344
11 345
11 346

12 391
12 392
12 393
12 394
12 395
12 396
12 397
12 398
12 399
12 400
12 401
12 402
12 403
12 404
12 405
12 406
12 407
12 408
12 409
12 410
12 411
12 412
12 413
12 414
12 415
12 416
12 417
12 418
12 419
12 420
12 421
12 422
12 423
12 424
12 425
12 426
12 427
12 428
12 429
12 430
12 431
12 432
12 433
12 434
12 435
12 436
12 437
12 438
12 439
12 440
12 441
12 442
12 443
12 444
12 445
12 446
12 447
12 448
12 449
12 450
12 451
12 452
12 453
12 454
12 455
12 456
12 457
12 458
12 459
12 460
12 461
12 462
12 463
12 464
12 465
12 466
12 467
12 468
12 469
12 470
12 471
12 472
12 473
12 474
12 475
12 476
12 477
12 478
12 479
12 480
12 481
12 482
12 483
12 484
12 485
12 486
12 487
12 488
12 489
12 490
12 491
12 492
12 493
12 494
12 495
12 496
12 497
12 498
12 499
12 500
12 501
12 502
12 503
12 504
12 505
12 506
12 507
12 508
12 509
12 510
12 511
12 512
12 513
12 514
12 515
12 516
12 517
12 518
12 519
12 520
12 521
12 522
12 523
12 524
12 525
12 526
12 527
12 528
12 529
12 530
12 531
12 532
12 533

13 578
13 579
13 580
13 581
13 582
13 583
13 584
13 585
13 586
13 587
13 588
13 589
13 590
13 591
13 592
13 593
13 594
13 595
13 596
13 597
13 598
13 599
13 600
13 601
13 602
13 603
13 604
13 605
13 606
13 607
13 608
13 609
13 610
13 611
13 612
13 613
13 614
13 615
13 616
13 617
13 618
13 619
13 620
13 621
13 622
13 623
13 624
13 625
13 626
13 627
13 628
13 629
13 630
13 631
13 632
13 633
13 634
13 635
13 636
13 637
13 638
13 639
13 640
13 641
13 642
13 643
13 644
13 645
13 646
13 647
13 648
13 649
13 650
13 651
13 652
13 653
13 654
13 655
13 656
13 657
13 658
13 659
13 660
13 661
13 662
13 663
13 664
13 665
13 666
13 667
13 668
13 669
13 670
13 671
13 672
13 673
13 674
13 675
13 676
13 677
13 678
13 679
13 680
13 681
13 682
13 683
13 684
13 685
13 686
13 687
13 688
13 689
13 690
13 691
13 692
13 693
13 694
13 695
13 696
13 697
13 698
13 699
13 700
13 701
13 702
13 703
13 704
13 705
13 706
13 707
13 708
13 709
13 710
13 711
13 712
13 713
13 714
13 715
13 716
13 717
13 718
13 719
13 720

14 765
14 766
14 767
14 768
14 769
14 770
14 771
14 772
14 773
14 774
14 775
14 776
14 777
14 778
14 779
14 780
14 781
14 782
14 783
14 784
14 785
14 786
14 787
14 788
14 789
14 790
14 791
14 792
14 793
14 794
14 795
14 796
14 797
14 798
14 799
14 800
14 801
14 802
14 803
14 804
14 805
14 806
14 807
14 808
14 809
14 810
14 811
14 812
14 813
14 814
14 815
14 816
14 817
14 818
14 819
14 820
14 821
14 822
14 823
14 824
14 825
14 826
14 827
14 828
14 829
14 830
14 831
14 832
14 833
14 834
14 835
14 836
14 837
14 838
14 839
14 840
14 841
14 842
14 843
14 844
14 845
14 846
14 847
14 848
14 849
14 850
14 851
14 852
14 853
14 854
14 855
14 856
14 857
14 858
14 859
14 860
14 861
14 862
14 863
14 864
14 865
14 866
14 867
14 868
14 869
14 870
14 871
14 872
14 873
14 874
14 875
14 876
14 877
14 878
14 879
14 880
14 881
14 882
14 883
14 884
14 885
14 886
14 887
14 888
14 889
14 890
14 891
14 892
14 893
14 894
14 895
14 896
14 897
14 898
14 899
14 900
14 901
14 902
14 903
14 904
14 905
14 906
14 907

15 952
15 953
15 954
15 955
15 956
15 957
15 958
15 959
15 960
15 961
15 962
15 963
15 964
15 965
15 966
15 967
15 968
15 969
15 970
15 971
15 972
15 973
15 974
15 975
15 976
15 977
15 978
15 979
15 980
15 981
15 982
15 983
15 984
15 985
15 986
15 987
15 988
15 989
15 990
15 991
15 992
15 993
15 994
15 995
15 996
15 997
15 998
15 999
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
16 16
16 17
16 18
16 19
16 20
16 21
16 22
16 23
16 24
16 25
16 26
16 27
16 28
16 29
16 30
16 31
16 32
16 33
16 34
16 35
16 36
16 37
16 38
16 39
16 40
16 41
16 42
16 43
16 44
16 45
16 46
16 47
16 48
16 49
16 50
16 51
16 52
16 53
16 54
16 55
16 56
16 57
16 58
16 59
16 60
16 61
16 62
16 63
16 64
16 65
16 66
16 67
16 68
16 69
16 70
16 71
16 72
16 73
16 74
16 75
16 76
16 77
16 78
16 79
16 80
16 81
16 82
16 83
16 84
16 85
16 86
16 87
16 88
16 89
16 90
16 91
16 92
16 93
16 94
16 95
16 96
16 97
16 98
16 99
16 100
16 101
16 102
16 103
16 104
16 105
16 106
16 107
16 108
16 109
16 110
16

17 156
17 157
17 158
17 159
17 160
17 161
17 162
17 163
17 164
17 165
17 166
17 167
17 168
17 169
17 170
17 171
17 172
17 173
17 174
17 175
17 176
17 177
17 178
17 179
17 180
17 181
17 182
17 183
17 184
17 185
17 186
17 187
17 188
17 189
17 190
17 191
17 192
17 193
17 194
17 195
17 196
17 197
17 198
17 199
17 200
17 201
17 202
17 203
17 204
17 205
17 206
17 207
17 208
17 209
17 210
17 211
17 212
17 213
17 214
17 215
17 216
17 217
17 218
17 219
17 220
17 221
17 222
17 223
17 224
17 225
17 226
17 227
17 228
17 229
17 230
17 231
17 232
17 233
17 234
17 235
17 236
17 237
17 238
17 239
17 240
17 241
17 242
17 243
17 244
17 245
17 246
17 247
17 248
17 249
17 250
17 251
17 252
17 253
17 254
17 255
17 256
17 257
17 258
17 259
17 260
17 261
17 262
17 263
17 264
17 265
17 266
17 267
17 268
17 269
17 270
17 271
17 272
17 273
17 274
17 275
17 276
17 277
17 278
17 279
17 280
17 281
17 282
17 283
17 284
17 285
17 286
17 287
17 288
17 289
17 290
17 291
17 292
17 293
17 294
17 295
17 296
17 297
17 298

18 343
18 344
18 345
18 346
18 347
18 348
18 349
18 350
18 351
18 352
18 353
18 354
18 355
18 356
18 357
18 358
18 359
18 360
18 361
18 362
18 363
18 364
18 365
18 366
18 367
18 368
18 369
18 370
18 371
18 372
18 373
18 374
18 375
18 376
18 377
18 378
18 379
18 380
18 381
18 382
18 383
18 384
18 385
18 386
18 387
18 388
18 389
18 390
18 391
18 392
18 393
18 394
18 395
18 396
18 397
18 398
18 399
18 400
18 401
18 402
18 403
18 404
18 405
18 406
18 407
18 408
18 409
18 410
18 411
18 412
18 413
18 414
18 415
18 416
18 417
18 418
18 419
18 420
18 421
18 422
18 423
18 424
18 425
18 426
18 427
18 428
18 429
18 430
18 431
18 432
18 433
18 434
18 435
18 436
18 437
18 438
18 439
18 440
18 441
18 442
18 443
18 444
18 445
18 446
18 447
18 448
18 449
18 450
18 451
18 452
18 453
18 454
18 455
18 456
18 457
18 458
18 459
18 460
18 461
18 462
18 463
18 464
18 465
18 466
18 467
18 468
18 469
18 470
18 471
18 472
18 473
18 474
18 475
18 476
18 477
18 478
18 479
18 480
18 481
18 482
18 483
18 484
18 485

In [8]:
conn.commit()

---

In [13]:
b = getInfoResponse("0000127517").text

In [14]:
xmlInfoObj = etree.fromstring(b.encode('utf-8'), parser=parser)

In [15]:
xmlInfoRoot = etree.ElementTree(xmlInfoObj).getroot()

In [16]:
etree.dump(xmlInfoRoot)

<facInfoList>
  <resultCode>000</resultCode>
  <resultMessage>조회 성공</resultMessage>
  <servList>
    <evalInfo>경보 및 피난설비,계단 또는 승강설비,대변기,복도,비치용품,일반사항,장애인전용주차구역,점자블록,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수</evalInfo>
    <faclNm>목면사무소</faclNm>
    <wfcltId>0000127517</wfcltId>
  </servList>
</facInfoList>


In [17]:
l2 = xmlInfoObj.find('.//evalInfo').text

In [18]:
type(l2)

str

In [5]:
type(cur.execute("SELECT evalInfo FROM facilities").fetchall())

list

In [11]:
cur.execute("SELECT evalInfo FROM facilities").fetchone()

('계단 또는 승강설비,대변기,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',)

In [29]:
keep = set({})
add = (1,2,4)
keep.update(add)
keep

{1, 2, 4}

In [60]:
s = set()
for row in cur.execute("SELECT evalInfo FROM facilities").fetchall():
    s.update(row)

In [72]:
sl = list(s)
sl

['대변기,소변기,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)',
 '경보 및 피난설비,계단 또는 승강설비,대변기,매표소, 판매기, 음료대,비치용품,일반사항,장애인전용주차구역,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',
 '점자블록,주출입구 높이차이 제거,주출입구 접근로',
 '대변기,비치용품,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)',
 '객실 및 침실,경보 및 피난설비,계단 또는 승강설비,관람석 및 열람석,대변기,매표소, 판매기, 음료대,복도,비치용품,세면대,소변기,유도 및 안내 설비,일반사항,점자블록,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',
 '경보 및 피난설비,대변기,일반사항,주출입구 접근로',
 '객실 및 침실,경보 및 피난설비,계단 또는 승강설비,관람석 및 열람석,대변기,복도,세면대,소변기,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',
 '객실 및 침실,경보 및 피난설비,계단 또는 승강설비,대변기,소변기,유도 및 안내 설비,장애인전용주차구역,점자블록,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',
 '계단 또는 승강설비,관람석 및 열람석,대변기,비치용품,세면대,소변기,일반사항,장애인전용주차구역,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',
 '경보 및 피난설비,대변기,일반사항,장애인전용주차구역,점자블록,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',
 '경보 및 피난설비,계단 또는 승강설비,대변기,매표소, 판매기, 음료대,복도,일반사항,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수',
 '경보 및 피난설비,대변기,매표소, 판매기, 음료대,복도,비치용품,일반사항,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구

In [75]:
sl[0]

'대변기,소변기,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)'

In [37]:
type(('객실, 침실, 경보'))

str

In [50]:
cur.execute("SELECT evalInfo FROM facilities").fetchall()[0][0].split(', ')

['계단 또는 승강설비,대변기,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수']

In [71]:
sll = []
for k in sl:
    sll.append(sl[k].split(', '))

TypeError: list indices must be integers or slices, not str

In [76]:
for m in cur.execute("SELECT * FROM facilities").fetchall():
    print(m)

(1, 20140610062, 37.562725711470605, 127.03256993326957, '(구)왕십리클리닉빌딩', 'UC0K02', '서울특별시 성동구 왕십리로 333', '0000073296', '계단 또는 승강설비,대변기,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(2, 20140615291, 37.47798704381922, 126.95328960225815, '(길성아파트)하이텔하우스', 'UC0U01', '서울특별시 관악구 관악로12길 16', '0000079520', '계단 또는 승강설비,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(3, 20140762484, 37.57393739290491, 126.9831873379122, '(재)대한불교계종유지재단', 'UC0A01', '서울특별시 종로구 우정국로 56', '0000000002', '주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(4, 20140512111, 37.485281715327524, 126.93135450484702, '(주)관전인베스트먼트', 'UC0L02', '서울특별시 관악구 신림로64길29 (신림동)', '0000142761', '객실 및 침실,계단 또는 승강설비,대변기,복도,세면대,소변기,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(5, 20140758598, 37.49076760950892, 126.98204810181372, '(주)대광메디텍', 'UC0A01', '서울특별시 동작구 동작대로35길 5', '0000000229', '주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(6, 20140511858, 37.48736471496628, 126.95322044449561, '(주)동성건설', 'UC0U04', '서울특별시 관악구 은천로35다길16 (봉천동)', '0000142677', '장애인전용주

(1839, 20140752552, 37.55086482953683, 127.12812700386782, '강동연세내과', 'UC0A14', '서울특별시 강동구 올림픽로  782', '0000002444', '계단 또는 승강설비,대변기,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(1840, 20140757845, 37.54731263907467, 127.12688067923253, '강동연세병원', 'UC0F02', '서울특별시 강동구 올림픽로 742', '0000006231', '객실 및 침실,경보 및 피난설비,계단 또는 승강설비,대변기,세면대,소변기,유도 및 안내 설비,일반사항,장애인전용주차구역,점자블록,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(1841, 20140763684, 37.53637037365218, 127.14893888008915, '강동요양치료데이케어센터', 'UC0H03', '서울특별시 강동구 천호대로 1243', '0000012380', '대변기,소변기,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(1842, 20140764608, 37.5363774663454, 127.14320473409843, '강동우체국', 'UC0A05', '서울특별시 강동구 양재대로112길 57', '0000001637', '경보 및 피난설비,계단 또는 승강설비,대변기,매표소, 판매기, 음료대,비치용품,일반사항,장애인전용주차구역,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(1843, 20140750517, 37.55890314189612, 127.16984477524556, '강동유정유치원', 'UC0G02', '서울특별시 강동구 상일로', '0000006770', '계단 또는 승강설비,대변기,소변기,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수

(3339, 20140509295, 37.558076571125, 126.9162306712027, '기정MAY아파트', 'UC0U01', '서울특별시 마포구 월드컵북로7길 26-6', '0000141115', '계단 또는 승강설비,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(3340, 20140615307, 37.55801874010831, 126.9159953273335, '기정MAY아파트', 'UC0U01', '서울특별시 마포구 월드컵북로7길 26-14', '0000079597', '계단 또는 승강설비,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(3341, 20140764532, 37.57741899090397, 127.01568590709009, '기타건축물 지하층 수석컴비지텔 주차장', 'UC0A12', '서울특별시 종로구 지봉로 82', '0000001926', '주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(3342, 20140764533, 37.57718112058052, 127.01574472443842, '기타건축물지하층 현대빌딩 주차장', 'UC0A12', '서울특별시 종로구 지봉로12길 3', '0000001928', '주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(3343, 20140749265, 37.47833456977137, 126.98927767751326, '기탄빌딩', 'UC0B01', '서울특별시 서초구 효령로 44', '0000004143', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(3344, 20140745290, 37.54610749368928, 127.14084247662126, '기풍하이빌', 'UC0U04', '서울특별시 강동구 상암로 164', '0000019147', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(3345, 2014076

(4838, 20140747563, 37.546539391942694, 126.84434183084002, '대림캐슬', 'UC0U04', '서울특별시 강서구 화곡로41길  14', '0000018824', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(4839, 20140744435, 37.535957427129, 126.85814474330999, '대림타운', 'UC0U04', '서울특별시 강서구 등촌로13다길  32-35', '0000018421', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(4840, 20140505230, 37.56993397552669, 126.93370832306006, '대림통상빌딩', 'UC0K02', '서울특별시 서대문구 연희로 142', '0000134968', '계단 또는 승강설비,대변기,일반사항,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(4841, 20140753506, 37.49343217192729, 126.89870320261335, '대림파출소', 'UC0A04', '서울특별시 영등포구 대림로23길 16-1', '0000001446', '경보 및 피난설비,계단 또는 승강설비,대변기,매표소, 판매기, 음료대,장애인전용주차구역,접수대 및 작업대,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(4842, 20140764085, 37.49617815310971, 126.89836928903225, '대림평화교회', 'UC0D01', '서울특별시 영등포구 대림로27가길 13', '0000005203', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(4843, 20140753166, 37.50252300233118, 126.89625715265586, '대림프라자', 'UC0G08', '서울특별시 영등포구 대림로  247', '0000009028', '계단 또는

(6338, 20140742760, 37.46008571439613, 126.90509141385242, '뜰안채', 'UC0U04', '서울특별시 금천구 독산로142길 35', '0000021260', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(6339, 20140746917, 37.47452694445095, 126.90487870766768, '뜰안채', 'UC0U04', '서울특별시 금천구 독산로94가길11 (독산동)', '0000021079', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(6340, 20140741637, 37.46086515894105, 126.9018667955733, '뜰안채', 'UC0U04', '서울특별시 금천구 시흥대로84길 48', '0000021291', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(6341, 20140742024, 37.485460035925904, 126.99314809732566, '띠아모르', 'UC0U04', '서울특별시 서초구 서초대로22길  38-5', '0000021869', '장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(6342, 20140606745, 37.54896171205411, 126.92092584269732, '라꼼마빌딩(서교동K빌딩)', 'UC0K02', '서울특별시 마포구 어울마당로  44-1', '0000072428', '계단 또는 승강설비,대변기,장애인전용주차구역,주출입구 높이차이 제거,주출입구 접근로,출입구(문),해당시설 층수')
(6343, 20140611713, 37.49765595023511, 127.03045182098198, '라누이 아모르 모텔', 'UC0L01', '서울특별시 강남구 테헤란로6길 14', '0000075870', '객실 및 침실,관람석 및 열람석,주출입구 높이차이 제거,주출입구 접근로,출입구(문)')
(6344, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

